In [1]:
print('hello')

hello


In [2]:
import os
%pwd

'/Users/lavish./Desktop/Deep Learning/End-to-End-Chest-cancer-Classification-Using-MLFLOW-DVC/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/lavish./Desktop/Deep Learning/End-to-End-Chest-cancer-Classification-Using-MLFLOW-DVC'

## Update Config.yaml

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path

## Update params.yaml

In [6]:
'''Note: params.yaml should not be empty otherwise it'll through error
    So, add some Dummy code : key:val'''

"Note: params.yaml should not be empty otherwise it'll through error\n    So, add some Dummy code : key:val"

## update constant, then
## update Configuration manager in src

In [7]:
from ChestCancerClassifier.constants import *
from ChestCancerClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
        

## Update the components

In [9]:
import gdown
import zipfile
import os
from ChestCancerClassifier import logger
from ChestCancerClassifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config


    def download_file(self)->str:
        '''
        Fetch data from the url
        '''
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs('artifacts/data_ingestion', exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        

    def extract_zipfile(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

## Update Pipeline

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e

[2024-10-21 14:38:40,398: INFO : common : yaml file: config/config.yaml loaded successfully]
[2024-10-21 14:38:40,400: INFO : common : yaml file: params.yaml loaded successfully]
[2024-10-21 14:38:40,402: INFO : common : created directory at: artifacts]
[2024-10-21 14:38:40,403: INFO : common : created directory at: artifacts/data_ingestion]
[2024-10-21 14:38:40,404: INFO : 2525879152 : Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=33e1a6c6-57a2-4106-bcee-f808ebd4f6eb
To: /Users/lavish./Desktop/Deep Learning/End-to-End-Chest-cancer-Classification-Using-MLFLOW-DVC/artifacts/data_ingestion/data.zip
100%|██████████| 49.0M/49.0M [00:05<00:00, 8.48MB/s]

[2024-10-21 14:38:51,525: INFO : 2525879152 : Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]
